In [2]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from lxml import html
from selenium.webdriver.common.keys import Keys
import time

df = pd.read_csv('vines.csv', header=None)
df.columns = ['vine', 'liter', 'cost']
df['vine'] = df['vine'].str.replace('\n', '')

In [3]:
df

,vine,liter,cost
0,Amour de Deutz Амур де Дейц RP 94/100,NaN,NaN
1,Amour de Deutz Rosé Амур де Дейц Розе,NaN,NaN
2,Amour de Deutz Rosé en étui Амур де Дейц Розе ...,NaN,NaN
3,Bastianich Vigne Orsone Pinot Grigio Бастианич...,NaN,NaN
4,Blanches Terres Premier Cru Brut Rose,"0,75","6 100,00"
...,...,...,...
115,Theodorine Brut Rose,"0,75","4 400,00"
116,Zonin Gran Cuvee Зонин Гран Кюве,NaN,NaN
117,Zonin Moscato delle Venezie Зонин Москато IGT,"0,75","2 050,00"
118,Zonin Prosecco DOC Зонин Просекко DOC,NaN,NaN


In [4]:

vines_names_lst = []
vines_find_lst = []
ratings_lst = []
liter_lst = []
cost_lst = []
avg_price_lst = []
ratings_count_lst = []
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.vivino.com')
for vine, liter, cost in zip(df['vine'], df['liter'], df['cost']):
    driver.find_element_by_class_name("searchBar__searchInput--2Nf0D").send_keys(vine)
    driver.find_element_by_class_name("searchBar__searchInput--2Nf0D").send_keys(Keys.ENTER)
    time.sleep(3)
    driver.find_element_by_class_name("searchBar__searchInput--2Nf0D").clear()
    source_data = driver.page_source
    soup = bs(source_data, 'lxml')
    name = soup.find_all('span', {'class':'bold'})[0].text.replace('\n', '')
    try:
        rating = float(soup.find_all('div', {'class':'text-inline-block light average__number'})[0].text.replace('\n', '').replace(',', '.'))
    except Exception:
        rating = None
    try:
        ratings_count = int(soup.find_all('p', {'class':'text-micro'})[0].text.replace('\n', '').replace(',', '.').replace(' ratings', ''))
    except Exception:
        ratings_count = None
    try:
        average_price = float(soup.find_all('span', {'class':'wine-price-value'})[0].text.replace('\n', '').replace(' ', '').replace(',', '.'))
    except Exception:
        try:
            href = 'https://www.vivino.com' + soup.find_all('span', {'class':'header-smaller text-block wine-card__name'})[0].a.attrs['href']
            driver.get(href)
            time.sleep(5)
            source_data = driver.page_source
            soup = bs(source_data, 'lxml')
            average_price = float(soup.find_all('span', {'class':'purchaseAvailabilityPPC__amount--2_4GT'})[0].text[:-1])
        except Exception as E:
            average_price = None
            print(E)
    liter_lst.append(liter)
    cost_lst.append(cost)
    vines_names_lst.append(vine)
    vines_find_lst.append(name)
    ratings_lst.append(rating)
    avg_price_lst.append(average_price)
    ratings_count_lst.append(ratings_count)
    print('|'+vine+'|', '|'+name+'|', rating, average_price, ratings_count)
    time.sleep(1)

In [ ]:
df_vines = pd.DataFrame()
df_vines['name'] = vines_names_lst
df_vines['find'] = vines_find_lst
df_vines['rating'] = ratings_lst
df_vines['avg_price'] = avg_price_lst
df_vines['ratings_count'] = ratings_count_lst
df_vines['liter'] = liter_lst
df_vines['cost'] = cost_lst
df_vines['rating'] = df_vines['rating'].astype(str).str.replace('.', ',')
df_vines['avg_price'] = df_vines['avg_price'].astype(str).str.replace('.', ',')
df_vines['ratings_count'] = df_vines['ratings_count'].astype(str).str.replace('.', ',')
df_vines.drop_duplicates(subset='find').to_csv('vines.csv', index=False)

In [5]:
df_vines = get_df_vines(df)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/elisejzelihovskij/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=89.0.4389.128)


In [340]:
df_vines

,name,find,rating,avg_price,ratings_count,liter,cost
0,Amour de Deutz Амур де Дейц RP 94/100,Deutz Amour de Deutz Millésime Brut Champagne,"4,5","44520,0","3820,0",NaN,NaN
1,Amour de Deutz Rosé Амур де Дейц Розе,Deutz Amour de Deutz Millesimé Rosé Brut Champ...,"4,5","29826,0","333,0",NaN,NaN
2,Amour de Deutz Rosé en étui Амур де Дейц Розе ...,Vitaliy Malanchuk Rose In . de Mix (РОЗЕ ІН ДЕ...,"4,5",nan,"3820,0",NaN,NaN
3,Bastianich Vigne Orsone Pinot Grigio Бастианич...,Bastianich Orsone Pinot Grigio,"3,7","1150,0","1081,0",NaN,NaN
4,Blanches Terres Premier Cru Brut Rose,Yann Alexandre Blanches Terres Brut Rosé Champ...,"3,7","6014,0","92,0","0,75","6 100,00"
...,...,...,...,...,...,...,...
115,Theodorine Brut Rose,Michel Loriot Théodorine Rosé Brut Champagne,"4,1","5900,0","181,0","0,75","4 400,00"
116,Zonin Gran Cuvee Зонин Гран Кюве,Zonin Gran Cuvée Extra Dry,"3,5",nan,"92,0",NaN,NaN
117,Zonin Moscato delle Venezie Зонин Москато IGT,Zonin Francesco Zonin Moscato delle Venezie,"4,3",nan,"1041,0","0,75","2 050,00"
118,Zonin Prosecco DOC Зонин Просекко DOC,Prosecco Doc Zonin,"3,8","372,0","10,0",NaN,NaN


In [341]:
df_vines = pd.DataFrame()
df_vines['name'] = vines_names_lst
df_vines['find'] = vines_find_lst
df_vines['rating'] = ratings_lst
df_vines['avg_price'] = avg_price_lst
df_vines['ratings_count'] = ratings_count_lst
df_vines['liter'] = liter_lst
df_vines['cost'] = cost_lst

In [342]:
df_vines['rating'] = df_vines['rating'].astype(str).str.replace('.', ',')
df_vines['avg_price'] = df_vines['avg_price'].astype(str).str.replace('.', ',')
df_vines['ratings_count'] = df_vines['ratings_count'].astype(str).str.replace('.', ',')

In [343]:
df_vines.drop_duplicates(subset='find').to_csv('vines.csv', index=False)

In [225]:
tst = df_vines.drop_duplicates(subset='find')

In [228]:
!pip install transliterate

In [264]:
from transliterate import translit

In [291]:
df_2 = pd.read_csv('vines_2.csv', header=None)

In [292]:
df_2.columns = ['id', 'vine', 'master', 'liter', 'cost']

In [294]:
df_2 = df_2.dropna(subset=['vine'])

In [295]:
df_2['vine'] = df_2['vine'].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [298]:
df_2['vine'] = df_2['vine'].apply((lambda x: translit(x, reversed=True)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [330]:
df_2[30:]

,id,vine,master,liter,cost
30,108027,Blek Tauer Red,Рех Кендерман ГмбХ Вайнкеллерай,"0,75","760,00"
31,123205,Bio Bio Shardone,Чело э Терра С.п.А.,"0,75","790,00"
32,124136,Bio Bio Merlo,Чело э Терра С.п.А.,"0,75","790,00"
33,115616,Gato Negro Pino Nuar,Винья Сан Педро Тарапака С.А.,"0,75","790,00"
34,118265,Gato Negro Shardone,Винья Сан Педро Тарапака С.А.,"0,75","790,00"
...,...,...,...,...,...
276,120287,Shampan' P'er Zhimone e Fis Enofil' Prem'e Krj...,Шампань Пьер Жимоне э Фис САС,"0,75","9 890,00"
277,124455,Vintazh,Шампань Луи Родерер,"0,75","10 990,00"
278,123471,Vintazh p/u grafika,Шампань Луи Родерер,"0,75","11 290,00"
279,123293,Roze p/u grafika,Шампань Луи Родерер,"0,75","11 290,00"


In [338]:
res = get_df_vines(df_2)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/elisejzelihovskij/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


|Kampo Delija La Mancha Ajren| |Campo Delia Airén| 3.4 443.0 1815
|Kampo Delija La Mancha Temprani| |Campo Delia Tempranillo| 3.3 443.0 2419
list index out of range
|TINI B'janko| |Banko Toni Teran - Borgonja| None None None
|TINI Rosso| |Tini Rosso| 3.5 620.0 1003
list index out of range
|Famil'ja de Felipe Tempranil'o| |Familia de Felipe Tempranillo| None None None
list index out of range
|Famil'ja de Felipe Viura| |Familia de Felipe Viura| None None None
|Trebb'jano Pino Gridzho novyj dizajn| |Biserno Il Pino di Biserno| 4.3 7550.0 12332
|Fiestino Brjut| |Casa Demonte Fiestino Brut| 3.5 670.0 362
|Fiestino Dol'che| |Casa Demonte Fiestino Dolce| 4.1 392.0 182
|Bruni Grekaniko Pino Gridzho| |Bruni Grecanico - Pinot Grigio| 3.5 500.0 3029
|TINI Roze (Emilija-Romanija)| |Tini Vino Rosso d'Italia| 3.4 600.0 3640
|Balaklava Shardone ZGU Krym| |Zolotaya Balka (Золотая Балка) Balaklava Loco Cimbali Barrique| 3.8 1212.0 670
list index out of range
|Balaklava Pino Nuar ZGU Krym| |PolOpuesto E

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".searchBar__searchInput--2Nf0D"}
  (Session info: chrome=89.0.4389.90)


In [339]:
df_2[30:]

,id,vine,master,liter,cost
30,108027,Blek Tauer Red,Рех Кендерман ГмбХ Вайнкеллерай,"0,75","760,00"
31,123205,Bio Bio Shardone,Чело э Терра С.п.А.,"0,75","790,00"
32,124136,Bio Bio Merlo,Чело э Терра С.п.А.,"0,75","790,00"
33,115616,Gato Negro Pino Nuar,Винья Сан Педро Тарапака С.А.,"0,75","790,00"
34,118265,Gato Negro Shardone,Винья Сан Педро Тарапака С.А.,"0,75","790,00"
...,...,...,...,...,...
276,120287,Shampan' P'er Zhimone e Fis Enofil' Prem'e Krj...,Шампань Пьер Жимоне э Фис САС,"0,75","9 890,00"
277,124455,Vintazh,Шампань Луи Родерер,"0,75","10 990,00"
278,123471,Vintazh p/u grafika,Шампань Луи Родерер,"0,75","11 290,00"
279,123293,Roze p/u grafika,Шампань Луи Родерер,"0,75","11 290,00"


In [334]:
res['rating'] = res['rating'].astype(str).str.replace('.', ',')
res['avg_price'] = res['avg_price'].astype(str).str.replace('.', ',')
res['ratings_count'] = res['ratings_count'].astype(str).str.replace('.', ',')

KeyError: 'rating'

In [319]:
summary = df_vines.append(res.drop_duplicates(subset='find'))

In [320]:
summary.to_csv('summary.csv', index=False)

In [322]:
res.to_csv('vines2.csv', index=False)